# TO DO
1. extract all text untill next example
2. clean the collected urls to only include lates version ( removes duplication)
https://ppubs.uspto.gov/pubwebapp/static/pages/ppubsbasic.html

# Imports

In [1]:
from collection import collect_urls,download_files,unzip_files
from utils import extract_examples_start_w_word,find_doc_number,process_siblings,extract_num_dot_examples,extract_examples_w_word,extract_experiments_w_heading,save_as_json,remove_duplicate_docs

# Data Collection

In [2]:
from lxml import etree
import requests

# url ="https://bulkdata.uspto.gov/data/patent/application/redbook/fulltext/2023/"
# rp = requests.get(url, timeout=10)
# parser = etree.XMLParser(recover=True)
# root = etree.fromstring(rp.text.encode(), parser)
# href_values = root.findall(".//a[@href]") 
# urls = [href.get('href') for href in href_values if href.get('href').endswith('.zip')]
# for ur in urls[:10]:
#     print(url + ur)

start_year = int(input("Enter the year you want to start from: "))
end_year = int(input("Enter the year you want to end at: "))


def fetch_urls_from_pto(start_year,end_year):
    urls = []
    for year in range(start_year, end_year + 1):
        url = f"https://bulkdata.uspto.gov/data/patent/application/redbook/fulltext/{year}/"
        rp = requests.get(url, timeout=10)
        parser = etree.XMLParser(recover=True)
        root = etree.fromstring(rp.text.encode(), parser)
        href_values = root.findall(".//a[@href]")
        urls.extend([href.get('href') for href in href_values if href.get('href').endswith('.zip')])
    return urls

urls = fetch_urls_from_pto(start_year,end_year)
print(len(urls))

In [6]:

import re
from collections import defaultdict
latest_versions = {}

# Regex to extract date and optional revision number
pattern = re.compile(r"(ipa\d{6})(?:_r(\d+))?\.zip")

for file in urls:
    match = pattern.match(file)
    if match:
        base_name, revision = match.groups()
        revision = int(revision) if revision else 0  # Default to 0 if no revision
        
        # Extract the current highest revision number for the base_name
        current_revision_match = re.search(r'_r(\d+)', latest_versions.get(base_name, ""))
        current_revision = int(current_revision_match.group(1)) if current_revision_match else 0
        
        # Update if the new file has a higher revision
        if base_name not in latest_versions or revision > current_revision:
            latest_versions[base_name] = file

# Get the final list of unique latest versions
latest_files = sorted(latest_versions.values())

print(len(latest_files))

156


In [14]:
def download_files(main_url, download_path,files):
    if not os.path.exists(download_path):
        os.makedirs(download_path)
    for index,file_name in enumerate(files):
        url = main_url + file_name


        zip_file_path = os.path.join(download_path, file_name) #.split(".")[-1]
        response = requests.get(url, stream=True, timeout=10)
        with open(zip_file_path, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"Downloaded {file_name} ------- {index + 1} / {len(files)}")
files = []
for year in range(start_year, end_year + 1):
        url = f"https://bulkdata.uspto.gov/data/patent/application/redbook/fulltext/{year}/"
        files.append(download_files(url, "patents_data", latest_files))

Downloaded ipa220106.zip ------- 1 / 156
Downloaded ipa220113.zip ------- 2 / 156
Downloaded ipa220120.zip ------- 3 / 156
Downloaded ipa220127.zip ------- 4 / 156
Downloaded ipa220203.zip ------- 5 / 156
Downloaded ipa220210.zip ------- 6 / 156
Downloaded ipa220217.zip ------- 7 / 156
Downloaded ipa220224.zip ------- 8 / 156
Downloaded ipa220303.zip ------- 9 / 156
Downloaded ipa220310.zip ------- 10 / 156
Downloaded ipa220317.zip ------- 11 / 156
Downloaded ipa220324.zip ------- 12 / 156
Downloaded ipa220331.zip ------- 13 / 156
Downloaded ipa220407.zip ------- 14 / 156
Downloaded ipa220414.zip ------- 15 / 156
Downloaded ipa220421.zip ------- 16 / 156
Downloaded ipa220428.zip ------- 17 / 156
Downloaded ipa220505.zip ------- 18 / 156
Downloaded ipa220512.zip ------- 19 / 156
Downloaded ipa220519.zip ------- 20 / 156
Downloaded ipa220526.zip ------- 21 / 156
Downloaded ipa220602.zip ------- 22 / 156
Downloaded ipa220609.zip ------- 23 / 156
Downloaded ipa220616.zip ------- 24 / 156
D

ConnectTimeout: HTTPSConnectionPool(host='bulkdata.uspto.gov', port=443): Max retries exceeded with url: /data/patent/application/redbook/fulltext/2022/ipa220804.zip (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001B7489FDCD0>, 'Connection to bulkdata.uspto.gov timed out. (connect timeout=10)'))

In [10]:
"yearly url: https://bulkdata.uspto.gov/data/patent/application/redbook/fulltext/2025/"
import requests
import os

def collect_urls(year):
    url = f"https://bulkdata.uspto.gov/data/patent/application/redbook/fulltext/{year}/"
    # params = {
    #     "data": f'{{"name":"APPXML","fromDate":"{start_date}","toDate":"{end_date}"}}'
    # }
    print(url)
    response = requests.get(url, timeout=10)
    if response.status_code != 200:
        print(f"Error: {response.status_code}")
        return []
    #print(f"Urls collected for the period from {start_date} to {end_date} successfully")
    return response.json()["productFiles"]

# start_date = input("Enter start date YYYY-MM: ")
# end_date = input("Enter end date YYYY-MM: ")
year = input("Enter year: ")
download_path = "patents_data"
unzip_path = "unzipped_patents_data"

try:
    os.makedirs(download_path, exist_ok=True)
    print(f"Download path set to: {download_path}")
except PermissionError:
    print("Error: No permission to create directory on D drive")
    # Fallback to current directory
    download_path = "patents_data"
    os.makedirs(download_path, exist_ok=True)
    print(f"Fallback: Download path set to: {download_path}")

files = collect_urls(year)
if files:
    total_size = sum(file["fileSize"] / 1024 / 1024 for file in files)
    print(
        f"\nTotal file size: {round(total_size / 1024, 2)} GB for  {year}\n"
    )
    download_files(files, download_path)
    unzip_files(download_path, unzip_path)
    

Download path set to: patents_data
https://bulkdata.uspto.gov/data/patent/application/redbook/fulltext/2023/


JSONDecodeError: Expecting value: line 3 column 1 (char 6)

In [2]:
unzip_path = "unzipped_patents_data"
download_path = "patents_data"
unzip_files(download_path, unzip_path)

Unzipping files:  12%|█▎        | 1/8 [00:08<01:01,  8.76s/it]

Unzipped ipa230105.zip to unzipped_patents_data


Unzipping files:  25%|██▌       | 2/8 [00:12<00:36,  6.05s/it]

Unzipped ipa230112.zip to unzipped_patents_data


Unzipping files:  38%|███▊      | 3/8 [00:15<00:21,  4.40s/it]

...

Unzipped ipa230126.zip to unzipped_patents_data


Unzipping files:  88%|████████▊ | 7/8 [00:53<00:08,  8.05s/it]

Unzipped ipa230126_r1.zip to unzipped_patents_data


Unzipping files: 100%|██████████| 8/8 [01:02<00:00,  7.77s/it]

Unzipped ipa230126_r2.zip to unzipped_patents_data


# Processing

In [3]:
import os
def process_xml_files(directory_path):
    """
    Process all XML files in the given directory and split them into parts.
    
    Args:
        directory_path (str): Path to directory containing XML files
        
    Returns:
        list: Combined list of XML parts from all files
    """
    all_xml_parts = []
    
    # Get all XML files in directory
    xml_files = [f for f in os.listdir(directory_path) if f.endswith('.xml')]
    
    # Process each file
    for xml_file in xml_files:
        file_path = os.path.join(directory_path, xml_file)
        try:
            with open(file_path, "r", encoding="utf-8") as file:
                content = file.read()
                parts = content.split('<?xml version="1.0" encoding="UTF-8"?>')
                # Remove empty parts and extend master list
                parts = [p for p in parts if p.strip()]
                all_xml_parts.extend(parts)
        except Exception as e:
            print(f"Error processing {xml_file}: {str(e)}")
    
    return all_xml_parts
xml_parts = process_xml_files("unzipped_patents_data")

In [4]:
len(xml_parts)

30085

In [2]:
# with open("unzipped_patents_data\\ipa230105.xml", "r", encoding="utf-8") as file:
#     content = file.read()
# xml_parts = content.split('<?xml version="1.0" encoding="UTF-8"?>')

In [3]:
xml_no_dup = remove_duplicate_docs(xml_parts)

In [3]:
# from bs4 import BeautifulSoup

# def extract_experiments_w_heading(text):
#     """Extracts the 'Examples' section and its experiments from a patent text."""

#     # Use BeautifulSoup to parse the structure (for HTML-like patents)
#     soup = BeautifulSoup(text, "html.parser")

#     # Find the "EXAMPLES" section heading
#     examples_heading = soup.find(
#         lambda tag: tag.name == "heading"
#         and (
#             "EXAMPLES" in tag.text.upper()
#             or "EXAMPLE" == tag.text.upper()
#             or "EXPERIMENT" == tag.text.upper()
#             or "EXPERIMENTS" in tag.text.upper()
#         )
#     )
#     if not examples_heading:
#         # print("No 'Examples' section found.")
#         return None


#     return examples_heading 

In [7]:

doc_w_exp = {}
i = 0
for xml in xml_no_dup:
    i += 1
    if i % 100 == 0:
        print(f"{i}/{len(xml_no_dup)} so far found {len(doc_w_exp)} docs with experiments")
    heading = extract_experiments_w_heading(xml)
    if heading:
        extracted_examples = []
        if len(heading) >1:
            for example in heading:
                extracted_ex_start_w_word = extract_examples_start_w_word(example.find_next_siblings())
                if len(extracted_ex_start_w_word) > 0:
                    extracted_examples.append(extracted_ex_start_w_word)
                else:
                    extracted_ex_w_word = process_siblings(example.find_next_siblings())
                    if extracted_ex_w_word:
                        if len(extracted_ex_w_word) > 0:
                            extracted_examples.append(extracted_ex_w_word)
                    else:
                        num_dot_examples = extract_num_dot_examples(str(heading[0].find_next_siblings()))
                        if len(num_dot_examples) > 0:
                            extracted_examples.append(num_dot_examples)
        elif len(heading) == 1:
            example_start_w = extract_examples_start_w_word(heading[0].find_next_siblings())
            if example_start_w:
                if len(example_start_w[0]["content"]) == 0:
                    extracted_ex_w_word = process_siblings(heading[0].find_next_siblings())
                    if len(extracted_ex_w_word) > 0:
                        extracted_examples.append(extracted_ex_w_word)
                else:
                    extracted_examples.append(example_start_w)
            else:
                extracted_ex_w_word = process_siblings(heading[0].find_next_siblings())
                if extracted_ex_w_word:
                    if len(extracted_ex_w_word) > 0:
                        extracted_examples.append(extracted_ex_w_word)
                else:
                    num_dot_examples = extract_num_dot_examples(str(heading[0].find_next_siblings()))
                    if len(num_dot_examples) > 0:
                        extracted_examples.append(num_dot_examples)
        else:
            extracted_ex_w_word = extract_examples_w_word(xml)
            if extracted_ex_w_word:
                extracted_examples.append(extracted_ex_w_word)
            # else:
            #     num_dot_examples = extract_num_dot_examples(str(heading[0].find_next_siblings()))
            #     extracted_examples.append(num_dot_examples)
    
        
    
        if len(extracted_examples) > 0:
            doc_w_exp[int(find_doc_number(xml)[0])] = extracted_examples
            # if len(doc_w_exp) == 100:
            #     break


"""100/8048 so far found 17 docs with experiments
200/8048 so far found 30 docs with experiments
300/8048 so far found 33 docs with experiments
400/8048 so far found 38 docs with experiments
500/8048 so far found 42 docs with experiments
600/8048 so far found 45 docs with experiments
700/8048 so far found 46 docs with experiments
800/8048 so far found 90 docs with experiments
900/8048 so far found 146 docs with experiments
1000/8048 so far found 187 docs with experiments
1100/8048 so far found 233 docs with experiments
1200/8048 so far found 251 docs with experiments
1300/8048 so far found 258 docs with experiments
1400/8048 so far found 263 docs with experiments
1500/8048 so far found 294 docs with experiments
1600/8048 so far found 298 docs with experiments
1700/8048 so far found 302 docs with experiments
1800/8048 so far found 320 docs with experiments
1900/8048 so far found 326 docs with experiments
2000/8048 so far found 329 docs with experiments
2100/8048 so far found 331 docs with experiments
2200/8048 so far found 332 docs with experiments
2300/8048 so far found 334 docs with experiments
2400/8048 so far found 361 docs with experiments
2500/8048 so far found 435 docs with experiments
with dup: 1196   out of 8048 in 12 minutes"""

"""100/7824 so far found 20 docs with experiments
200/7824 so far found 34 docs with experiments
300/7824 so far found 36 docs with experiments
400/7824 so far found 42 docs with experiments
500/7824 so far found 45 docs with experiments
600/7824 so far found 49 docs with experiments
700/7824 so far found 51 docs with experiments
800/7824 so far found 98 docs with experiments
900/7824 so far found 162 docs with experiments
1000/7824 so far found 217 docs with experiments
1100/7824 so far found 253 docs with experiments
1200/7824 so far found 261 docs with experiments
1300/7824 so far found 266 docs with experiments
1400/7824 so far found 295 docs with experiments
1500/7824 so far found 304 docs with experiments
1600/7824 so far found 309 docs with experiments
1700/7824 so far found 328 docs with experiments
1800/7824 so far found 333 docs with experiments
1900/7824 so far found 339 docs with experiments
2000/7824 so far found 342 docs with experiments
2100/7824 so far found 343 docs with experiments
2200/7824 so far found 345 docs with experiments
2300/7824 so far found 358 docs with experiments
2400/7824 so far found 439 docs with experiments
2500/7824 so far found 510 docs with experiments
no dup 949  out of 7824 in 13.5 minutes"""

100/8048 so far found 19 docs with experiments
200/8048 so far found 33 docs with experiments
300/8048 so far found 36 docs with experiments
400/8048 so far found 41 docs with experiments
500/8048 so far found 45 docs with experiments
600/8048 so far found 48 docs with experiments
700/8048 so far found 49 docs with experiments
800/8048 so far found 94 docs with experiments
900/8048 so far found 151 docs with experiments
1000/8048 so far found 192 docs with experiments
1100/8048 so far found 238 docs with experiments
1200/8048 so far found 257 docs with experiments
1300/8048 so far found 264 docs with experiments
1400/8048 so far found 269 docs with experiments
1500/8048 so far found 303 docs with experiments
1600/8048 so far found 308 docs with experiments
1700/8048 so far found 313 docs with experiments
1800/8048 so far found 331 docs with experiments
1900/8048 so far found 337 docs with experiments
2000/8048 so far found 342 docs with experiments
2100/8048 so far found 345 docs with 

'100/7824 so far found 20 docs with experiments\n200/7824 so far found 34 docs with experiments\n300/7824 so far found 36 docs with experiments\n400/7824 so far found 42 docs with experiments\n500/7824 so far found 45 docs with experiments\n600/7824 so far found 49 docs with experiments\n700/7824 so far found 51 docs with experiments\n800/7824 so far found 98 docs with experiments\n900/7824 so far found 162 docs with experiments\n1000/7824 so far found 217 docs with experiments\n1100/7824 so far found 253 docs with experiments\n1200/7824 so far found 261 docs with experiments\n1300/7824 so far found 266 docs with experiments\n1400/7824 so far found 295 docs with experiments\n1500/7824 so far found 304 docs with experiments\n1600/7824 so far found 309 docs with experiments\n1700/7824 so far found 328 docs with experiments\n1800/7824 so far found 333 docs with experiments\n1900/7824 so far found 339 docs with experiments\n2000/7824 so far found 342 docs with experiments\n2100/7824 so far

In [8]:
save_as_json(doc_w_exp,filename="test_no_dup.json")

Saved as 957_w_dup.json


In [11]:
def process_document_examples(xml_parts, max_docs=100):
    """Process and extract examples from XML documents.
    
    Args:
        xml_parts (list): List of XML document parts
        max_docs (int): Maximum number of documents to process
        
    Returns:
        dict: Document numbers mapped to extracted examples
    """
    doc_with_examples = {}
    
    for xml in xml_parts:
        example_headings = extract_experiments_w_heading(xml)
        if not example_headings:
            continue
            
        extracted_examples = []
        
        try:
            if len(example_headings) > 1:
                # Process multiple headings
                for heading in example_headings:
                    siblings = heading.find_next_siblings()
                    examples = extract_examples_start_w_word(siblings)
                    
                    if examples:
                        extracted_examples.append(examples)
                    else:
                        processed = process_siblings(siblings)
                        if processed:
                            extracted_examples.append(processed)
                            
            else:
                # Process single heading
                siblings = example_headings[0].find_next_siblings()
                examples = extract_examples_start_w_word(siblings)
                
                if examples and examples[0]["content"]:
                    extracted_examples.append(examples)
                else:
                    processed = process_siblings(siblings)
                    if processed:
                        extracted_examples.append(processed)
            
            # Fall back to direct XML processing if needed
            if not extracted_examples:
                examples = extract_examples_w_word(xml)
                if examples:
                    extracted_examples.append(examples)
            
            # Store results if examples found
            if extracted_examples:
                doc_number = int(find_doc_number(xml)[0])
                doc_with_examples[doc_number] = extracted_examples
                
                if len(doc_with_examples) >= max_docs:
                    break
                    
        except Exception as e:
            print(f"Error processing document: {str(e)}")
            continue
            
    return doc_with_examples
